# Build and Train

Now we're ready to begin building our sentiment classifier and begin training. We will be building out what is essentially a *frame* around Bert, that will allow us to perform language classification. First, we can initialize the Bert model, which we will load as a pretrained model from transformers.

In [1]:
from transformers import TFAutoModel

2022-11-12 22:48:59.251174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-12 22:48:59.462958: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-12 22:48:59.876755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-12 22:48:59.876821: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

2022-11-12 22:49:01.942730: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-12 22:49:01.972680: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-12 22:49:01.973041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-12 22:49:01.973669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [3]:
# we can view the model using the summary method
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


Now we need to define the frame around Bert, we need:

* **Two** input layers (one for input IDs and one for attention mask).

* A post-bert dropout layer to reduce the likelihood of overfitting and improve generalization.

* Max pooling layer to convert the 3D tensors output by Bert to 2D.

* Final output activations using softmax for outputting categorical probabilities.

In [4]:
import tensorflow as tf

In [5]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), 
                                  name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), 
                             name='attention_mask', dtype='int32')

# Transformer - we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access final activations with [0]

# Classifier head
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings) # Rectified linear units
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x) # Use softmax to create probability distribution across the 5 labels

### We can now define our model, specifying input and output layers. Finally, we freeze the Bert layer because Bert is already highly trained, and contains a huge number of parameters so will take a very long time to train further. Nonetheless, if you'd like to train Bert too, there is nothing wrong with doing so.

In [6]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [7]:
# To freeze layers (i.e. not re-train them)
model.layers[2].trainable = False

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

### Our model architecture is now setup, and we can initialize our training parameters like so:



In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss = loss, metrics = [acc])

### Now all we need to do is train our model. For this, we need to load in our training and validation datasets - which also requires our dataset element specs to be defined.

In [10]:
element_spec=({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 
               'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, 
              tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [11]:
# load the training and validation sets
train_ds = tf.data.Dataset.load('train', element_spec=element_spec)
val_ds = tf.data.Dataset.load('val', element_spec=element_spec)

# view the input format
train_ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

We can now define our model, specifying input and output layers. Finally, we freeze the Bert layer because Bert is already highly trained, and contains a huge number of parameters so will take a very long time to train further. Nonetheless, if you'd like to train Bert too, there is nothing wrong with doing so.

### And now we train our model as usual.

In [12]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

8778/8778 [==============================] - 2445s 278ms/step - loss: 1.1137 - accuracy: 0.5521 - val_loss: 1.0646 - val_accuracy: 0.5704


### Finally, we save our model!

In [13]:
model.save('sentiment_model')

INFO:tensorflow:Assets written to: sentiment_model/assets


INFO:tensorflow:Assets written to: sentiment_model/assets
